## 一、项目背景
假如你训练了一个目标检测模型，你想让朋友快速体验一下，你会怎么做呢？
一般来讲你会把代码打包发给朋友，然后告诉他怎么配环境，怎么部署模型。或者连同环境封装成docker镜像，让对方直接运行实例，但是对方得懂docker，也要懂模型部署等知识。即便朋友愿意去花时间体验，一波操作下来会面临各种奇怪的问题。

为了方便快捷的体验应用，AI Studio平台最新上线了Gradio在线部署环境。不需要各种麻烦的操作，只需一键导入图片，即可立即生成效果图。

本项目以[PaddleDetection-PPYOLOE训练好的COCO模型](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.6/configs/ppyoloe/README_cn.md)为例，介绍如何使用Gradio和FastDeploy实现目标检测应用部署。

![](https://ai-studio-static-online.cdn.bcebos.com/6b49b31ac9d947839bba93dee974e00d1e1250fd28dd464caa954e990c7c095f)




> 以下为COCO数据集的前部分类别与对应标号，文中部署好了COCO物体检测应用，大家可以体验一下。

![](https://ai-studio-static-online.cdn.bcebos.com/82853ae705e64a8b8bb55bc9389a8a15b591ff86cc5f4eb186588abffae3455b)

## 二、方法步骤

### 2.1训练一个目标检测模型
 要想部署模型，首先得训练一个模型，本文是直接下载的[PaddleDetection-PPYOLOE训练好的COCO模型](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.6/configs/ppyoloe/README_cn.md)，当然也可以参考官方的一些项目进行训练。用PaddleDetection训练好的模型包含三个部分，.pdparams文件为模型权重，只需取 **.pdparams** 文件即可。为方便演示，我把下载好的 **.pdparams** 文件添加到根目录里。

![](https://ai-studio-static-online.cdn.bcebos.com/27565c9e4ccb4d93babc00a69222d209120ada7f0bc34f1fbe25f1c03fbfb7db)



### 2.2导出部署模型
模型训练好之后，我们要通过一些命令导出部署模型，注意：**训练的模型文件和部署的模型文件不同**。

部署模型主要用于推理，具体包括四种文件：

```
ppyoloe_crn_l_300e_coco    # 模型目录
├── infer_cfg.yml      # 模型配置信息
├── model.pdiparams     # 参数文件
└── model.pdiparams.info # 模型信息
└── model.pdmodel     # 模型文件

```

由于是目标检测任务，我们使用paddledetection套件以及一些命令导出部署模型，参考链接：https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.6/deploy/EXPORT_MODEL.md

由于网络缓慢，这里直接解压PaddleDetection-release-2.6版本

In [ ]:
# 解压方式安装PaddleDetection
!unzip data/data207390/PaddleDetection-release-2.6.zip
!mv PaddleDetection-release-2.6 PaddleDetection

In [ ]:
# 用命令安装PaddleDetection
# !git clone https://gitee.com/paddlepaddle/PaddleDetection.git

In [4]:
%cd PaddleDetection

/home/aistudio/PaddleDetection


In [ ]:
# 安装依赖
!pip install -r requirements.txt --user
!python setup.py install --user

In [6]:
# 导出部署模型
# 指定训练配置文件
# 指定训练好的模型文件
!python tools/export_model.py \
-c configs/ppyoloe/ppyoloe_plus_crn_l_80e_coco.yml \
-o weights=../ppyoloe_plus_crn_l_80e_coco.pdparams

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
[05/11 11:46:36] ppdet.utils.checkpoint INFO: Finish loading model weights: ../ppyoloe_plus_crn_l_80e_coco.pdparams
[05/11 11:46:37] ppdet.data.source.category

### 2.3fastdeploy验证

In [ ]:
# 安装相关包
!pip install numpy opencv-python fastdeploy-python -f https://www.paddlepaddle.org.cn/whl/fastdeploy.html --user

**注意**：直接运行下一步可能会出现报错 **No module named 'fastdeploy'** ，这时，我们需要**先保存文件**，然后重启内核，再运行下一步


![](https://ai-studio-static-online.cdn.bcebos.com/0bc0a3b146f04d4c82ed775bec37aa53377a01bf51b74c31882f0e161e4bb76f)
![](https://ai-studio-static-online.cdn.bcebos.com/1b748db264f74cbd8053f7b469c0c97d0a5bc5c6b250443084e80a840f7c3ce8)


In [1]:
# 验证部署是否正确
import cv2
import fastdeploy.vision as vision
model = vision.detection.PPYOLOE("/home/aistudio/PaddleDetection/output_inference/ppyoloe_plus_crn_l_80e_coco/model.pdmodel",
                                    "/home/aistudio/PaddleDetection/output_inference/ppyoloe_plus_crn_l_80e_coco/model.pdiparams", 
                                    "/home/aistudio/PaddleDetection/output_inference/ppyoloe_plus_crn_l_80e_coco/infer_cfg.yml", 
                                )
im = cv2.imread("/home/aistudio/mm.jpg")
result = model.predict(im)
vis_im = vision.vis_detection(im, result, score_threshold=0.5)
cv2.imwrite("/home/aistudio/work/result.png", vis_im)

[INFO] fastdeploy/vision/common/processors/transform.cc(45)::FuseNormalizeCast	Normalize and Cast are fused to Normalize in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(93)::FuseNormalizeHWC2CHW	Normalize and HWC2CHW are fused to NormalizeAndPermute  in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(159)::FuseNormalizeColorConvert	BGR2RGB and NormalizeAndPermute are fused to NormalizeAndPermute with swap_rb=1
[INFO] fastdeploy/runtime/backends/openvino/ov_backend.cc(218)::InitFromPaddle	number of streams:1.
[INFO] fastdeploy/runtime/backends/openvino/ov_backend.cc(228)::InitFromPaddle	affinity:YES.
[INFO] fastdeploy/runtime/backends/openvino/ov_backend.cc(240)::InitFromPaddle	Compile OpenVINO model on device_name:CPU.
[INFO] fastdeploy/runtime/runtime.cc(279)::CreateOpenVINOBackend	Runtime initialized with Backend::OPENVINO in Device::CPU.


True

## 三、Gradio在线预览

1. 打开gradio应用，系统会自动生成一个`untitled.gradio.py`文件
2. 将本项目的`demo.gradio.py`替换掉生成的`untitled.gradio.py`文件，在AI Studio上，gradio应用统一需要以`{filename}.gradio.py`命名，平台会自动识别。
3. 在根目录新建一个`launch`文件夹，将.gradio.py文件、output_inference文件夹、example图像（如有）、requirements.txt文件**一并放入**launch文件夹内
4. 如果是自己导出的模型，需要在demo.gradio.py修改model为自己的导出模型，打开demo.gradio.py查看效果，无报错在进行下一步

    ```
    >>cd ./launch/
    ├── demo.gradio.py
    ├── output_inference
    │   ├── infer_cfg.yml      
    │   ├── model.pdiparams
    │   ├── model.pdiparams.info
    │   ├── model.pdmodel
    ├── example.jpg
    ├── requirements.txt
    ```



![](https://ai-studio-static-online.cdn.bcebos.com/802fd1968212433e9d33d98e1a2ae9998a3b9a41cda74cc8a37d800f5b5350aa)


### 3.2FastDeploy介绍

FastDeploy是一款全场景、易用灵活、极致高效的AI推理部署工具。提供超过 🔥160+ Text，Vision， Speech和跨模态模型📦开箱即用的部署体验。包括物体检测、字符识别（OCR）、人脸、人像扣图、多目标跟踪系统、NLP、Stable Diffusion文图生成、TTS 等几十种任务场景。FastDeploy需要安装一些包进行适配，安装包在requirements.txt文件夹里，部署时会自动安装。大家可以尝试一下FastDeploy支持的其他模型进行应用部署。

![](https://ai-studio-static-online.cdn.bcebos.com/4f8821ec97f9458f81d013fea830853dba348c04842349628c694adf923f0a7e)

> 参数：
> * model_file(str): 模型文件路径
> * params_file(str): 参数文件路径
> * config_file(str): 推理配置yaml文件路径

> FastDeploy的PPYOLOE Python部署参考链接：https://github.com/PaddlePaddle/FastDeploy/blob/develop/docs/cn/quick_start/models/python.md

### 3.2gradio.py文件介绍

Gradio的核心是它的`gr.Interface`函数，用来构建可视化界面。

- fn：放你用来处理的函数，这里的fn即为**FastDeploy预测过程**，只有几行代码
- inputs：写你的输入类型，这里输入的是图像，所以是"image"
- outputs：写你的输出类型，这里输出的是图像，所以是"image"

最后我们用`demo.lauch()`把页面一发布，一个本地静态交互页面就完成了。

![](https://ai-studio-static-online.cdn.bcebos.com/fadbf8e8f80844cc81927ff369242819e7742fde76e44e21a89b9cb89302399e)


## 四、应用部署
 * 将`bushu.gradio.py`放入launch文件夹内，点击应用部署，右边出现报错，不要在意。`bushu.gradio.py`文件与`demo.gradio.py`主要区别在于model路径
 
 * 这里我选择CPU部署，因为不需要算力，如果选择GPU则需要更改FastDeploy的安装包，以及调用命令，可参考上边的文档
![](https://ai-studio-static-online.cdn.bcebos.com/c6146170280e4083acb4c82dc44b0881934ea16e96794a69b9876ca9d5801939)

![](https://ai-studio-static-online.cdn.bcebos.com/e79cd7cb044e4e87ac0e7e305eb96e3d2ad9f95e1f88413eb5e8276d845422c2)



## 五、参考资料

- [RT-DETR：全新ViT系列模型助力铁路工人安全作业[Gradio版本发布！]](https://aistudio.baidu.com/aistudio/projectdetail/6000200)
- [Gradio上线！手把手教你搭建在线AI应用：以视频字幕提取为例](https://aistudio.baidu.com/aistudio/projectdetail/5779911)
- [FastDeploy](https://github.com/PaddlePaddle/FastDeploy/tree/develop)